# AutoKaggle: Iterative Debugging and Testing

## Mục Tiêu

Notebook này tập trung nghiên cứu sâu về quy trình gỡ lỗi và kiểm thử lặp lại (iterative debugging and testing) được giới thiệu trong bài báo "AutoKaggle: A Multi-Agent Framework for Autonomous Data Science Competitions". Quy trình này đóng vai trò quan trọng trong việc đảm bảo tính chính xác và độ tin cậy của code được tạo ra bởi các mô hình ngôn ngữ lớn (LLMs) trong bối cảnh các tác vụ khoa học dữ liệu phức tạp.

Sau khi hoàn thành notebook này, bạn sẽ:
1. Hiểu rõ quy trình gỡ lỗi và kiểm thử lặp lại trong AutoKaggle
2. Nắm được ba công cụ chính được sử dụng trong quá trình này: thực thi code, gỡ lỗi và kiểm thử đơn vị
3. Hiểu được tầm quan trọng của kiểm thử đơn vị trong các tác vụ khoa học dữ liệu
4. Có thể triển khai một quy trình gỡ lỗi và kiểm thử lặp lại sử dụng Python

## Trích Xuất Từ Bài Báo

> "In AutoKaggle, the Developer adopts a development approach based on iterative error correction and testing. It ensures the robustness and correctness of generated code through iterative execution, debugging, and testing." (Section 3.2, Page 4)

> "Figure 2 shows the overall process of iterative debugging and testing. Specifically, the Developer first generates code based on the current state st, the plan Pφi created by the Planner, and the historical context H: Cφi = GenerateCode(st, Pφi, H). Cφi is the generated code for phase φi, and GenerateCode(·) represents the code generation function executed by the Developer. The historical context H includes previous phases' code, outputs, and other relevant information from other agents' activities." (Section 3.2, Page 4-5)

> "After the initial code generation, it enters an iterative debugging and testing process. This process can be described by Algorithm 2." (Section 3.2, Page 5)

> "Developer utilize three primary tools: code execution, code debugging, and unit testing." (Section 3.2, Page 5)

![Iterative Debugging and Testing](https://m-a-p.ai/AutoKaggle.github.io/assets/images/fig2.png)
*(Note: Diagram representation - actual image not included)*

## Cài Đặt Môi Trường

In [ ]:
!pip install -q langchain langchain-openai langchain-core
!pip install -q pandas numpy scikit-learn

In [ ]:
import os
import sys
import json
import traceback
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Tuple, Any, Union
import unittest
from io import StringIO
from contextlib import redirect_stdout, redirect_stderr
from enum import Enum

# Import LangChain components
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Set up environment variables
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # Replace with your actual key

## Phần 1: Phân Tích Quy Trình Gỡ Lỗi và Kiểm Thử Lặp Lại

Quy trình gỡ lỗi và kiểm thử lặp lại trong AutoKaggle là một phần quan trọng giúp đảm bảo code được tạo ra đáp ứng các yêu cầu và có thể tin cậy được. Hãy phân tích các thành phần chính của quy trình này:

### Giải Thích Lý Thuyết

#### 1. Quy Trình Tổng Quát

Quy trình gỡ lỗi và kiểm thử lặp lại có thể được tóm tắt như sau:

1. **Tạo Code Ban Đầu**: Developer tạo code dựa trên:
   - Trạng thái hiện tại (st)
   - Kế hoạch được tạo bởi Planner (Pφi)
   - Ngữ cảnh lịch sử (H)

2. **Vòng Lặp Gỡ Lỗi và Kiểm Thử**:
   - Thực thi code và kiểm tra lỗi
   - Nếu có lỗi, gỡ lỗi và tạo phiên bản sửa đổi
   - Nếu không có lỗi, chạy kiểm thử đơn vị
   - Nếu kiểm thử thất bại, gỡ lỗi và sửa chữa
   - Lặp lại cho đến khi code chạy không lỗi và vượt qua tất cả các kiểm thử

3. **Xử Lý Lỗi Lặp Lại**:
   - Theo dõi lịch sử lỗi
   - Nếu gặp lỗi tương tự nhiều lần, tạo lại code hoàn toàn

4. **Giới Hạn Số Lần Thử**:
   - Đặt giới hạn tối đa cho số lần thử gỡ lỗi
   - Nếu vượt quá giới hạn, đánh dấu giai đoạn là thất bại

#### 2. Ba Công Cụ Chính

Developer sử dụng ba công cụ chính trong quy trình:

**a) Code Execution (Thực Thi Code)**
- Chạy code và bắt lỗi runtime
- Ghi lại thông báo lỗi khi phát hiện
- Cung cấp đầu vào cho công cụ gỡ lỗi

**b) Code Debugging (Gỡ Lỗi Code)**
- Phân tích thông báo lỗi
- Định vị vị trí lỗi trong code
- Tạo bản sửa lỗi
- Thực hiện quy trình gỡ lỗi 3 bước:
  1. Xác định vị trí lỗi
  2. Sửa lỗi
  3. Kết hợp đoạn code đúng và đã sửa lỗi

**c) Unit Testing (Kiểm Thử Đơn Vị)**
- Chạy các kiểm thử đã định nghĩa trước
- Đảm bảo code không chỉ chạy mà còn đáp ứng các yêu cầu
- Kiểm tra tính nhất quán và logic của code

#### 3. Tầm Quan Trọng của Kiểm Thử Đơn Vị

Kiểm thử đơn vị đóng vai trò đặc biệt quan trọng trong quy trình này, vì:

- Đảm bảo tính chính xác về mặt logic của code, không chỉ đơn thuần là không có lỗi cú pháp
- Ngăn chặn lỗi từ giai đoạn trước lan truyền sang giai đoạn sau
- Đảm bảo code đáp ứng đúng mục tiêu và yêu cầu của giai đoạn
- Cung cấp phản hồi cụ thể về vấn đề trong code, giúp quá trình gỡ lỗi hiệu quả hơn

Như bài báo đã nêu: "Trong các tác vụ phức tạp và đòi hỏi độ chính xác cao như các cuộc thi khoa học dữ liệu Kaggle, việc chỉ đảm bảo code chạy không lỗi là không đủ... Cần thiết kế các kiểm thử đơn vị tỉ mỉ không chỉ để xác minh tính đúng đắn của code mà còn để đảm bảo nó đáp ứng các tiêu chuẩn logic và hiệu suất dự kiến."

## Phần 2: Triển Khai Mã Nguồn Cho Quy Trình Gỡ Lỗi và Kiểm Thử Lặp Lại

Dưới đây, chúng ta sẽ triển khai các thành phần chính của quy trình gỡ lỗi và kiểm thử lặp lại:

In [ ]:
# Triển khai Developer Agent với khả năng gỡ lỗi
class DeveloperAgent:
    def __init__(self, model: BaseChatModel):
        """
        Khởi tạo Developer Agent với mô hình ngôn ngữ lớn
        
        Parameters:
        - model: Mô hình ngôn ngữ lớn dùng cho tác tử
        """
        self.model = model
        self.system_prompt_generate = """
        Bạn là một Developer Agent chuyên nghiệp trong việc viết code cho các tác vụ khoa học dữ liệu.
        
        Nhiệm vụ của bạn là viết code Python dựa trên kế hoạch đã được cung cấp. Code của bạn phải:
        1. Cấu trúc tốt, hiệu quả và chính xác
        2. Tuân thủ nghiêm ngặt kế hoạch đã đề ra
        3. Bao gồm xử lý lỗi và kiểm tra đầu vào
        4. Dễ đọc và có đủ chú thích cho các phần phức tạp
        5. Có khả năng thực thi trực tiếp
        
        Tập trung vào việc tạo code chất lượng cao để đáp ứng các yêu cầu cụ thể của giai đoạn hiện tại.
        """
        
        self.system_prompt_debug = """
        Bạn là một Developer Agent chuyên nghiệp trong việc gỡ lỗi code cho các tác vụ khoa học dữ liệu.
        
        Nhiệm vụ của bạn là phân tích và sửa lỗi trong code Python đã được cung cấp. Quy trình gỡ lỗi của bạn phải:
        1. Phân tích kỹ lưỡng thông báo lỗi để hiểu nguyên nhân
        2. Xác định chính xác vị trí lỗi trong code
        3. Đề xuất và triển khai sửa lỗi
        4. Đảm bảo sửa lỗi không tạo ra vấn đề mới
        5. Trả về phiên bản code hoàn chỉnh đã sửa lỗi (không chỉ đoạn cần sửa)
        
        Dựa trên kinh nghiệm gỡ lỗi code khoa học dữ liệu, hãy tạo phiên bản sửa đổi đảm bảo code hoạt động đúng.
        """
        
        self.generate_prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt_generate),
            MessagesPlaceholder(variable_name="history"),
            HumanMessage(content="{input}")
        ])
        
        self.debug_prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=self.system_prompt_debug),
            MessagesPlaceholder(variable_name="history"),
            HumanMessage(content="{input}")
        ])
        
        self.generate_chain = self.generate_prompt | self.model | StrOutputParser()
        self.debug_chain = self.debug_prompt | self.model | StrOutputParser()
    
    def generate_code(self, phase_name: str, plan: str) -> str:
        """
        Tạo code dựa trên kế hoạch đã được cung cấp
        
        Parameters:
        - phase_name: Tên của giai đoạn hiện tại
        - plan: Kế hoạch chi tiết cho giai đoạn
        
        Returns:
        - Code đã tạo
        """
        input_text = f"""# Generate Code for Phase: {phase_name}
        
        ## Plan
        {plan}
        
        Please implement Python code to execute this plan. Ensure your code is well-structured, efficient, and includes appropriate error handling.
        Return only the Python code without explanations.
        """
        
        result = self.generate_chain.invoke({"input": input_text, "history": []})
        
        # Trích xuất code từ kết quả (nếu có định dạng markdown)
        if "```python" in result and "```" in result:
            code_blocks = []
            in_code_block = False
            for line in result.split("\n"):
                if line.strip() == "```python" or line.strip() == "```python3":
                    in_code_block = True
                    continue
                elif line.strip() == "```" and in_code_block:
                    in_code_block = False
                    continue
                elif in_code_block:
                    code_blocks.append(line)
            
            if code_blocks:
                return "\n".join(code_blocks)
        
        return result
    
    def debug_code(self, phase_name: str, code: str, error_message: str) -> str:
        """
        Gỡ lỗi code dựa trên thông báo lỗi
        
        Parameters:
        - phase_name: Tên của giai đoạn hiện tại
        - code: Code cần gỡ lỗi
        - error_message: Thông báo lỗi
        
        Returns:
        - Code đã sửa lỗi
        """
        input_text = f"""# Debug Code for Phase: {phase_name}
        
        ## Code with Error
        ```python
        {code}
        ```
        
        ## Error Message
        ```
        {error_message}
        ```
        
        Please debug the code above. Analyze the error message, identify the root cause, and provide the complete fixed version of the code.
        Return only the corrected Python code without explanations.
        """
        
        result = self.debug_chain.invoke({"input": input_text, "history": []})
        
        # Trích xuất code từ kết quả (nếu có định dạng markdown)
        if "```python" in result and "```" in result:
            code_blocks = []
            in_code_block = False
            for line in result.split("\n"):
                if line.strip() == "```python" or line.strip() == "```python3":
                    in_code_block = True
                    continue
                elif line.strip() == "```" and in_code_block:
                    in_code_block = False
                    continue
                elif in_code_block:
                    code_blocks.append(line)
            
            if code_blocks:
                return "\n".join(code_blocks)
        
        return result

In [ ]:
# Công cụ thực thi code
class CodeExecutor:
    @staticmethod
    def execute_code(code: str, globals_dict: Dict = None, locals_dict: Dict = None) -> Tuple[bool, str, Any]:
        """
        Thực thi code Python và bắt lỗi
        
        Parameters:
        - code: Mã Python cần thực thi
        - globals_dict: Dictionary cho global namespace
        - locals_dict: Dictionary cho local namespace
        
        Returns:
        - Tuple gồm (success_flag, output/error_message, result_object)
        """
        if globals_dict is None:
            globals_dict = {}
        if locals_dict is None:
            locals_dict = {}
            
        # Thêm các thư viện thông dụng vào globals
        globals_dict.update({
            "pd": pd,
            "np": np,
            "__builtins__": __builtins__
        })
        
        # Bắt đầu capture output
        stdout_buffer = StringIO()
        stderr_buffer = StringIO()
        result = None
        success = False
        
        try:
            with redirect_stdout(stdout_buffer), redirect_stderr(stderr_buffer):
                # Thực thi code như một khối lệnh
                exec(code, globals_dict, locals_dict)
                success = True
                
        except Exception as e:
            # Bắt và xử lý lỗi
            error_type = type(e).__name__
            error_msg = str(e)
            error_traceback = traceback.format_exc()
            stderr_buffer.write(f"{error_type}: {error_msg}\n{error_traceback}")
            
        # Lấy output
        stdout_output = stdout_buffer.getvalue()
        stderr_output = stderr_buffer.getvalue()
        
        # Kết hợp output
        if success:
            combined_output = stdout_output
            if stderr_output:  # Có thể có warnings
                combined_output += "\nWarnings:\n" + stderr_output
        else:
            combined_output = stderr_output
            
        return success, combined_output, locals_dict

In [ ]:
# Triển khai công cụ kiểm thử đơn vị
class UnitTestRunner:
    @staticmethod
    def create_test_case(test_name: str, test_description: str, test_function: str) -> str:
        """
        Tạo một test case trong unittest
        
        Parameters:
        - test_name: Tên của test case
        - test_description: Mô tả về mục đích của test
        - test_function: Code của hàm kiểm thử
        
        Returns:
        - String chứa code test case hoàn chỉnh
        """
        template = f"""
class {test_name}(unittest.TestCase):
    """{test_description}"""
    
    {test_function}
        """
        return template
    
    @staticmethod
    def run_tests(code: str, test_cases: List[Dict]) -> Tuple[bool, Dict]:
        """
        Chạy các kiểm thử đơn vị trên code
        
        Parameters:
        - code: Code cần kiểm thử
        - test_cases: Danh sách các test case với format {"name": str, "description": str, "function": str}
        
        Returns:
        - Tuple gồm (all_tests_passed, test_results)
        """
        # Tạo môi trường thực thi riêng biệt
        globals_dict = {}
        
        # Thêm thư viện unittest
        globals_dict["unittest"] = unittest
        globals_dict["pd"] = pd
        globals_dict["np"] = np
        
        # Thực thi code ban đầu trong môi trường
        success, output, locals_dict = CodeExecutor.execute_code(code, globals_dict)
        
        if not success:
            return False, {"error": output}
        
        # Kết hợp globals và locals từ quá trình thực thi
        globals_dict.update(locals_dict)
        
        # Tạo và thêm các test case
        test_suite = unittest.TestSuite()
        test_results = {}
        
        for test_case in test_cases:
            test_name = test_case["name"]
            test_description = test_case["description"]
            test_function = test_case["function"]
            
            # Tạo và thực thi test case
            test_code = UnitTestRunner.create_test_case(test_name, test_description, test_function)
            
            try:
                exec(test_code, globals_dict)
                test_case_class = globals_dict[test_name]
                test_suite.addTest(unittest.makeSuite(test_case_class))
            except Exception as e:
                test_results[test_name] = {
                    "passed": False,
                    "error": f"Error creating test case: {type(e).__name__}: {str(e)}"
                }
                continue
        
        # Chạy các test
        test_runner = unittest.TextTestRunner(stream=StringIO())
        test_result = test_runner.run(test_suite)
        
        # Phân tích kết quả
        all_passed = test_result.wasSuccessful()
        
        for test_case in test_cases:
            test_name = test_case["name"]
            if test_name not in test_results:  # Chưa có lỗi khi tạo test
                passed = True
                error_message = ""
                
                # Kiểm tra các test failure
                for failure in test_result.failures + test_result.errors:
                    if test_name in str(failure[0]):
                        passed = False
                        error_message = failure[1]
                        break
                
                test_results[test_name] = {
                    "passed": passed,
                    "error": error_message if not passed else ""
                }
        
        return all_passed, test_results

In [ ]:
# Triển khai quy trình gỡ lỗi và kiểm thử lặp lại
class IterativeDebugger:
    def __init__(self, developer: DeveloperAgent, max_attempts: int = 5, error_threshold: int = 3):
        """
        Khởi tạo IterativeDebugger
        
        Parameters:
        - developer: Tác tử Developer dùng để tạo và gỡ lỗi code
        - max_attempts: Số lần thử tối đa
        - error_threshold: Ngưỡng lỗi để quyết định tạo lại code
        """
        self.developer = developer
        self.max_attempts = max_attempts
        self.error_threshold = error_threshold
        self.error_history = []
    
    def debug_and_test(self, phase_name: str, plan: str, test_cases: List[Dict]) -> Tuple[str, bool, str, Dict]:
        """
        Thực hiện quy trình gỡ lỗi và kiểm thử lặp lại
        
        Parameters:
        - phase_name: Tên của giai đoạn hiện tại
        - plan: Kế hoạch cho giai đoạn
        - test_cases: Danh sách các test case
        
        Returns:
        - Tuple (final_code, success_flag, execution_output, test_results)
        """
        round = 0
        execution_flag = False
        retry_flag = False
        self.error_history = []
        
        # Tạo code ban đầu
        current_code = self.developer.generate_code(phase_name, plan)
        execution_output = ""
        test_results = {}
        
        # Vòng lặp gỡ lỗi và kiểm thử
        while round < self.max_attempts:
            # Nếu đang thử lại hoặc mới bắt đầu
            if round == 0 or retry_flag:
                current_code = self.developer.generate_code(phase_name, plan)
                self.error_history = []
                retry_flag = False
            
            # Thực thi code
            globals_dict = {}
            success, execution_output, _ = CodeExecutor.execute_code(current_code, globals_dict)
            
            if not success:
                # Code có lỗi thực thi
                self.error_history.append(execution_output)
                
                # Kiểm tra xem có nên tạo lại code không
                if len(self.error_history) >= self.error_threshold:
                    # Kiểm tra xem lỗi có lặp lại không
                    unique_errors = set([error.split('\n')[0] for error in self.error_history[-self.error_threshold:]])
                    if len(unique_errors) == 1:  # Cùng một lỗi lặp lại
                        retry_flag = True
                        continue
                
                # Gỡ lỗi code
                current_code = self.developer.debug_code(phase_name, current_code, execution_output)
            else:
                # Code thực thi thành công, chạy unit tests
                all_tests_passed, test_results = UnitTestRunner.run_tests(current_code, test_cases)
                
                if all_tests_passed:
                    # Tất cả tests đều pass
                    execution_flag = True
                    break
                else:
                    # Có tests không pass
                    error_msg = "Unit tests failed: " + json.dumps(test_results, indent=2)
                    self.error_history.append(error_msg)
                    current_code = self.developer.debug_code(phase_name, current_code, error_msg)
            
            round += 1
        
        return current_code, execution_flag, execution_output, test_results
    
    def evaluate_retry(self, error_history: List[str]) -> bool:
        """
        Đánh giá xem có nên tạo lại code từ đầu không
        
        Parameters:
        - error_history: Lịch sử các thông báo lỗi
        
        Returns:
        - True nếu nên tạo lại, False nếu không
        """
        if len(error_history) < self.error_threshold:
            return False
        
        # Lấy n lỗi gần nhất
        recent_errors = error_history[-self.error_threshold:]
        
        # Trích xuất phần đầu của thông báo lỗi (thường là loại lỗi)
        error_types = [error.split('\n')[0] for error in recent_errors]
        
        # Nếu cùng loại lỗi xuất hiện nhiều lần liên tiếp
        if len(set(error_types)) == 1:  # Tất cả đều giống nhau
            return True
        
        return False

## Phần 3: Thí Nghiệm Với Quy Trình Gỡ Lỗi và Kiểm Thử

Hãy tạo một thí nghiệm để minh họa quy trình gỡ lỗi và kiểm thử lặp lại. Chúng ta sẽ sử dụng một bài toán đơn giản trong giai đoạn làm sạch dữ liệu (Data Cleaning):

In [ ]:
# Dữ liệu mẫu
def create_sample_data():
    """
    Tạo dữ liệu mẫu cho thí nghiệm
    
    Returns:
    - DataFrame chứa dữ liệu mẫu với các vấn đề cần làm sạch
    """
    np.random.seed(42)
    
    # Tạo dữ liệu cơ bản
    data = {
        'ID': range(1, 101),
        'Name': [f"Person_{i}" for i in range(1, 101)],
        'Age': np.random.randint(18, 80, 100),
        'Income': np.random.normal(50000, 15000, 100),
        'Education': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], 100),
        'Married': np.random.choice(['Yes', 'No'], 100),
        'Children': np.random.randint(0, 5, 100)
    }
    
    df = pd.DataFrame(data)
    
    # Thêm các vấn đề cần làm sạch
    # 1. Giá trị thiếu
    missing_indices = np.random.choice(df.index, 20, replace=False)
    df.loc[missing_indices[:5], 'Age'] = np.nan
    df.loc[missing_indices[5:10], 'Income'] = np.nan
    df.loc[missing_indices[10:15], 'Education'] = np.nan
    df.loc[missing_indices[15:], 'Children'] = np.nan
    
    # 2. Giá trị ngoại lai
    outlier_indices = np.random.choice(df.index, 10, replace=False)
    df.loc[outlier_indices[:5], 'Age'] = np.random.choice([5, 8, 12, 100, 120], 5)  # Tuổi không hợp lý
    df.loc[outlier_indices[5:], 'Income'] = np.random.choice([500, 1000, 5000, 500000, 1000000], 5)  # Thu nhập không hợp lý
    
    # 3. Dữ liệu trùng lặp
    duplicate_indices = np.random.choice(df.index, 5, replace=False)
    for idx in duplicate_indices:
        duplicate_row = df.loc[idx].copy()
        df = pd.concat([df, pd.DataFrame([duplicate_row])], ignore_index=True)
    
    # 4. Lỗi định dạng
    format_error_indices = np.random.choice(df.index, 10, replace=False)
    df.loc[format_error_indices[:5], 'Education'] = df.loc[format_error_indices[:5], 'Education'].apply(lambda x: x.lower() if isinstance(x, str) else x)
    df.loc[format_error_indices[5:], 'Married'] = df.loc[format_error_indices[5:], 'Married'].apply(lambda x: x.lower() if isinstance(x, str) else x)
    
    return df

# Tạo dữ liệu mẫu
sample_df = create_sample_data()

# Hiển thị dữ liệu mẫu để kiểm tra
sample_df.head()

In [ ]:
# Kế hoạch cho giai đoạn Data Cleaning
data_cleaning_plan = """
# Data Cleaning Plan

## Task 1: Handle Missing Values
- Objective: Fill or remove missing values in the dataset
- Methods:
  * For Age column: Fill with median age
  * For Income column: Fill with mean income
  * For Education column: Fill with mode (most frequent value)
  * For Children column: Fill with 0 (assuming no information means no children)
- Expected Output: DataFrame with no missing values
- Constraints: Must preserve the original index

## Task 2: Handle Outliers
- Objective: Detect and handle outliers in numerical columns
- Methods:
  * Use Z-score method with threshold of 3.0
  * For Age column: Clip values to valid range (18-80)
  * For Income column: Clip values to range (10000-100000)
- Expected Output: DataFrame with outliers handled
- Constraints: Must preserve the number of rows

## Task 3: Remove Duplicates
- Objective: Identify and remove duplicate records
- Methods:
  * Consider all columns except 'ID' when identifying duplicates
  * Keep the first occurrence of each duplicate set
- Expected Output: DataFrame with no duplicate records
- Constraints: Must report the number of duplicates removed

## Task 4: Standardize Categorical Values
- Objective: Ensure consistent formatting for categorical columns
- Methods:
  * For Education column: Capitalize the first letter
  * For Married column: Convert all values to uppercase (YES/NO)
- Expected Output: DataFrame with standardized categorical values
- Constraints: Must not alter the meaning of the data
"""

# Test cases cho quá trình kiểm thử
data_cleaning_test_cases = [
    {
        "name": "TestMissingValues",
        "description": "Test that all missing values are properly handled",
        "function": """
def test_no_missing_values(self):
    # Kiểm tra xem cleaned_df có tồn tại không
    self.assertTrue('cleaned_df' in globals(), "cleaned_df không được tìm thấy trong môi trường")
    
    # Lấy cleaned_df từ globals
    df = globals()['cleaned_df']
    
    # Kiểm tra số lượng giá trị thiếu
    missing_values = df.isnull().sum().sum()
    self.assertEqual(missing_values, 0, f"Còn {missing_values} giá trị thiếu trong dữ liệu")
        """
    },
    {
        "name": "TestOutliers",
        "description": "Test that outliers are properly handled",
        "function": """
def test_age_in_range(self):
    # Kiểm tra xem cleaned_df có tồn tại không
    self.assertTrue('cleaned_df' in globals(), "cleaned_df không được tìm thấy trong môi trường")
    
    # Lấy cleaned_df từ globals
    df = globals()['cleaned_df']
    
    # Kiểm tra giá trị Age có nằm trong khoảng hợp lý không
    self.assertTrue((df['Age'] >= 18).all() and (df['Age'] <= 80).all(), 
                   "Có giá trị Age nằm ngoài khoảng hợp lý 18-80")
    
def test_income_in_range(self):
    # Kiểm tra xem cleaned_df có tồn tại không
    self.assertTrue('cleaned_df' in globals(), "cleaned_df không được tìm thấy trong môi trường")
    
    # Lấy cleaned_df từ globals
    df = globals()['cleaned_df']
    
    # Kiểm tra giá trị Income có nằm trong khoảng hợp lý không
    self.assertTrue((df['Income'] >= 10000).all() and (df['Income'] <= 100000).all(), 
                   "Có giá trị Income nằm ngoài khoảng hợp lý 10000-100000")
        """
    },
    {
        "name": "TestDuplicates",
        "description": "Test that duplicates are removed",
        "function": """
def test_no_duplicates(self):
    # Kiểm tra xem cleaned_df có tồn tại không
    self.assertTrue('cleaned_df' in globals(), "cleaned_df không được tìm thấy trong môi trường")
    
    # Lấy cleaned_df từ globals
    df = globals()['cleaned_df']
    
    # Kiểm tra không có bản ghi trùng lặp (không tính ID)
    columns_to_check = [col for col in df.columns if col != 'ID']
    duplicates = df.duplicated(subset=columns_to_check).sum()
    self.assertEqual(duplicates, 0, f"Còn {duplicates} bản ghi trùng lặp trong dữ liệu")
        """
    },
    {
        "name": "TestCategoricalValues",
        "description": "Test that categorical values are standardized",
        "function": """
def test_education_format(self):
    # Kiểm tra xem cleaned_df có tồn tại không
    self.assertTrue('cleaned_df' in globals(), "cleaned_df không được tìm thấy trong môi trường")
    
    # Lấy cleaned_df từ globals
    df = globals()['cleaned_df']
    
    # Kiểm tra format của giá trị Education
    education_values = df['Education'].dropna().unique()
    for value in education_values:
        self.assertEqual(value, value.capitalize(), 
                       f"Giá trị Education '{value}' không đúng format (chữ cái đầu viết hoa)")
    
def test_married_format(self):
    # Kiểm tra xem cleaned_df có tồn tại không
    self.assertTrue('cleaned_df' in globals(), "cleaned_df không được tìm thấy trong môi trường")
    
    # Lấy cleaned_df từ globals
    df = globals()['cleaned_df']
    
    # Kiểm tra format của giá trị Married
    married_values = df['Married'].dropna().unique()
    for value in married_values:
        self.assertEqual(value, value.upper(), 
                       f"Giá trị Married '{value}' không đúng format (chữ in hoa)")
        """
    }
]

### Trình Diễn Quy Trình Lặp Đi Lặp Lại

Dưới đây, chúng ta sẽ mô phỏng quá trình gỡ lỗi và kiểm thử lặp lại, giả sử có một số lỗi trong code ban đầu:

In [ ]:
# Mô phỏng quá trình gỡ lỗi và kiểm thử lặp lại
def demonstrate_iterative_debugging():
    """
    Mô phỏng quy trình gỡ lỗi và kiểm thử lặp lại
    
    Returns:
    - Dictionary chứa thông tin về quá trình và kết quả
    """
    # Khởi tạo mô hình LLM
    # Lưu ý: Trong môi trường thực, bạn sẽ sử dụng mô hình thực tế thay vì mô phỏng
    model = ChatOpenAI(model="gpt-4o")  # Thay bằng mô hình thực khi chạy
    
    # Khởi tạo Developer và IterativeDebugger
    developer = DeveloperAgent(model)
    debugger = IterativeDebugger(developer)
    
    # Theo dõi quá trình
    process_log = [
        {
            "step": "Initial Code Generation",
            "description": "Developer tạo code dựa trên kế hoạch làm sạch dữ liệu",
            "code": """
import pandas as pd
import numpy as np

def clean_data(df):
    """Clean the dataset by handling missing values, outliers, duplicates, and standardizing categorical values."""
    # Create a copy of the dataframe to avoid modifying the original
    cleaned_df = df.copy()
    
    # Task 1: Handle Missing Values
    # For Age column: Fill with median age
    cleaned_df['Age'].fillna(cleaned_df['Age'].median(), inplace=True)
    
    # For Income column: Fill with mean income
    cleaned_df['Income'].fillna(cleaned_df['Income'].mean(), inplace=True)
    
    # For Education column: Fill with mode (most frequent value)
    mode_education = cleaned_df['Education'].mode()[0]
    cleaned_df['Education'].fillna(mode_education, inplace=True)
    
    # For Children column: Fill with 0
    cleaned_df['Children'].fillna(0, inplace=True)
    
    # Task 2: Handle Outliers
    # For Age column: Clip values to valid range (18-80)
    cleaned_df['Age'] = cleaned_df['Age'].clip(18, 80)
    
    # For Income column: Clip values to range (10000-100000)
    cleaned_df['Income'] = cleaned_df['Income'].clip(10000, 100000)
    
    # Task 3: Remove Duplicates
    # Consider all columns except 'ID' when identifying duplicates
    columns_to_check = [col for col in cleaned_df.columns if col != 'ID']
    duplicates_count = cleaned_df.duplicated(subset=columns_to_check).sum()
    cleaned_df.drop_duplicates(subset=columns_to_check, keep='first', inplace=True)
    print(f"Removed {duplicates_count} duplicate records.")
    
    # Task 4: Standardize Categorical Values
    # For Education column: Capitalize the first letter
    # Note: There's a bug here, should be cleaned_df['Education'] = cleaned_df['Education'].str.capitalize()
    cleaned_df['Education'] = cleaned_df['Education'].apply(lambda x: x.capitalize() if isinstance(x, str) else x)
    
    # For Married column: Convert all values to uppercase (YES/NO)
    # Note: There's a bug here, missing the check for string type before applying upper()
    cleaned_df['Married'] = cleaned_df['Married'].upper()
    
    return cleaned_df

# Apply the cleaning function to the sample dataframe
# Note: Missing the global variable assignment that tests expect
clean_data(sample_df)
"""
        }
    ]
    
    # Lần chạy đầu tiên - Có lỗi runtime
    execution_success, execution_output, _ = CodeExecutor.execute_code(
        process_log[0]["code"], 
        {"sample_df": sample_df}
    )
    
    process_log.append({
        "step": "First Execution",
        "description": "Thực thi code ban đầu",
        "success": execution_success,
        "output": execution_output
    })
    
    # Lần gỡ lỗi đầu tiên - Sửa lỗi 'upper()' không áp dụng được cho Series
    first_debug = """
import pandas as pd
import numpy as np

def clean_data(df):
    """Clean the dataset by handling missing values, outliers, duplicates, and standardizing categorical values."""
    # Create a copy of the dataframe to avoid modifying the original
    cleaned_df = df.copy()
    
    # Task 1: Handle Missing Values
    # For Age column: Fill with median age
    cleaned_df['Age'].fillna(cleaned_df['Age'].median(), inplace=True)
    
    # For Income column: Fill with mean income
    cleaned_df['Income'].fillna(cleaned_df['Income'].mean(), inplace=True)
    
    # For Education column: Fill with mode (most frequent value)
    mode_education = cleaned_df['Education'].mode()[0]
    cleaned_df['Education'].fillna(mode_education, inplace=True)
    
    # For Children column: Fill with 0
    cleaned_df['Children'].fillna(0, inplace=True)
    
    # Task 2: Handle Outliers
    # For Age column: Clip values to valid range (18-80)
    cleaned_df['Age'] = cleaned_df['Age'].clip(18, 80)
    
    # For Income column: Clip values to range (10000-100000)
    cleaned_df['Income'] = cleaned_df['Income'].clip(10000, 100000)
    
    # Task 3: Remove Duplicates
    # Consider all columns except 'ID' when identifying duplicates
    columns_to_check = [col for col in cleaned_df.columns if col != 'ID']
    duplicates_count = cleaned_df.duplicated(subset=columns_to_check).sum()
    cleaned_df.drop_duplicates(subset=columns_to_check, keep='first', inplace=True)
    print(f"Removed {duplicates_count} duplicate records.")
    
    # Task 4: Standardize Categorical Values
    # For Education column: Capitalize the first letter
    cleaned_df['Education'] = cleaned_df['Education'].apply(lambda x: x.capitalize() if isinstance(x, str) else x)
    
    # For Married column: Convert all values to uppercase (YES/NO)
    # Fixed bug: Added str method for Series and type checking for each value
    cleaned_df['Married'] = cleaned_df['Married'].apply(lambda x: x.upper() if isinstance(x, str) else x)
    
    return cleaned_df

# Apply the cleaning function to the sample dataframe
# Note: Still missing the global variable assignment that tests expect
clean_data(sample_df)
"""
    
    process_log.append({
        "step": "First Debug",
        "description": "Developer sửa lỗi về phương thức upper() cho Series",
        "code": first_debug
    })
    
    # Lần chạy thứ hai - Thành công nhưng kiểm thử thất bại
    execution_success, execution_output, _ = CodeExecutor.execute_code(
        process_log[2]["code"], 
        {"sample_df": sample_df}
    )
    
    process_log.append({
        "step": "Second Execution",
        "description": "Thực thi code sau lần gỡ lỗi đầu tiên",
        "success": execution_success,
        "output": execution_output
    })
    
    # Chạy kiểm thử đơn vị - Thất bại do thiếu biến toàn cục
    all_tests_passed, test_results = UnitTestRunner.run_tests(
        process_log[2]["code"], 
        data_cleaning_test_cases
    )
    
    process_log.append({
        "step": "First Unit Testing",
        "description": "Chạy kiểm thử đơn vị trên code đã sửa lỗi",
        "success": all_tests_passed,
        "results": test_results
    })
    
    # Lần gỡ lỗi thứ hai - Sửa lỗi thiếu biến toàn cục
    second_debug = """
import pandas as pd
import numpy as np

def clean_data(df):
    """Clean the dataset by handling missing values, outliers, duplicates, and standardizing categorical values."""
    # Create a copy of the dataframe to avoid modifying the original
    cleaned_df = df.copy()
    
    # Task 1: Handle Missing Values
    # For Age column: Fill with median age
    cleaned_df['Age'].fillna(cleaned_df['Age'].median(), inplace=True)
    
    # For Income column: Fill with mean income
    cleaned_df['Income'].fillna(cleaned_df['Income'].mean(), inplace=True)
    
    # For Education column: Fill with mode (most frequent value)
    mode_education = cleaned_df['Education'].mode()[0]
    cleaned_df['Education'].fillna(mode_education, inplace=True)
    
    # For Children column: Fill with 0
    cleaned_df['Children'].fillna(0, inplace=True)
    
    # Task 2: Handle Outliers
    # For Age column: Clip values to valid range (18-80)
    cleaned_df['Age'] = cleaned_df['Age'].clip(18, 80)
    
    # For Income column: Clip values to range (10000-100000)
    cleaned_df['Income'] = cleaned_df['Income'].clip(10000, 100000)
    
    # Task 3: Remove Duplicates
    # Consider all columns except 'ID' when identifying duplicates
    columns_to_check = [col for col in cleaned_df.columns if col != 'ID']
    duplicates_count = cleaned_df.duplicated(subset=columns_to_check).sum()
    cleaned_df.drop_duplicates(subset=columns_to_check, keep='first', inplace=True)
    print(f"Removed {duplicates_count} duplicate records.")
    
    # Task 4: Standardize Categorical Values
    # For Education column: Capitalize the first letter
    cleaned_df['Education'] = cleaned_df['Education'].apply(lambda x: x.capitalize() if isinstance(x, str) else x)
    
    # For Married column: Convert all values to uppercase (YES/NO)
    cleaned_df['Married'] = cleaned_df['Married'].apply(lambda x: x.upper() if isinstance(x, str) else x)
    
    return cleaned_df

# Apply the cleaning function to the sample dataframe
# Fixed bug: Add global variable assignment for unit tests
cleaned_df = clean_data(sample_df)
"""
    
    process_log.append({
        "step": "Second Debug",
        "description": "Developer sửa lỗi về biến toàn cục cleaned_df",
        "code": second_debug
    })
    
    # Lần chạy thứ ba - Thành công và kiểm thử thành công
    execution_success, execution_output, _ = CodeExecutor.execute_code(
        process_log[5]["code"], 
        {"sample_df": sample_df}
    )
    
    process_log.append({
        "step": "Final Execution",
        "description": "Thực thi code cuối cùng",
        "success": execution_success,
        "output": execution_output
    })
    
    # Chạy kiểm thử đơn vị lần cuối
    all_tests_passed, test_results = UnitTestRunner.run_tests(
        process_log[5]["code"], 
        data_cleaning_test_cases
    )
    
    process_log.append({
        "step": "Final Unit Testing",
        "description": "Chạy kiểm thử đơn vị trên code cuối cùng",
        "success": all_tests_passed,
        "results": test_results
    })
    
    return {
        "process_log": process_log,
        "final_success": all_tests_passed,
        "iterations": len(process_log) // 2  # Số lần lặp lại (mỗi lần gồm thực thi và gỡ lỗi)
    }

# Chạy mô phỏng
# demonstration_results = demonstrate_iterative_debugging()

## Phân Tích Thuật Toán

### Giải Thích Thuật Toán Gỡ Lỗi và Kiểm Thử Lặp Lại

Dưới đây là mã giả mô tả thuật toán gỡ lỗi và kiểm thử lặp lại từ bài báo (Algorithm 2):

```
ALGORITHM: Development based on Iterative Debugging and Testing
INPUT: Initial code Cφi, Current state st, Plan Pφi, Historical context H, Maximum tries max_tries, Error threshold threshold
OUTPUT: Debugged and tested code C'φi, Execution flag execution_flag

1. round ← 0
2. error_flag ← false
3. execution_flag ← true
4. retry_flag ← false
5. error_history ← ∅
6. while round < max_tries do
7.     if round = 0 or retry_flag then
8.         Cφi ← GenerateCode(st, Pφi, H)
9.         error_history ← ∅
10.        retry_flag ← false
11.    error_flag, Eφi ← RunCode(Cφi)
12.    if error_flag then
13.        error_history ← error_history ∪ {Eφi}
14.        if |error_history| >= threshold then
15.            retry_flag ← EvaluateRetry(error_history)
16.        if retry_flag then
17.            continue
18.        Cφi ← DebugCode(Cφi, Eφi, H)
19.    else
20.        Rφi ← ExecuteUnitTests(Cφi, Tφi)
21.        if ∃rj ∈ Rφi : rj = 0 then
22.            Cφi ← DebugTestFailures(Cφi, Rφi, H)
23.        else
24.            execution_flag ← true
25.            break
26.    round ← round + 1
27. if round = max_tries then
28.     execution_flag ← false
29. return Cφi, execution_flag
```

### Phân Tích Độ Phức Tạp

**Độ phức tạp thời gian**:
- Trong trường hợp xấu nhất, thuật toán thực hiện `max_tries` lần lặp
- Mỗi lần lặp có thể bao gồm GenerateCode, RunCode, EvaluateRetry, DebugCode, ExecuteUnitTests, và DebugTestFailures
- GenerateCode và DebugCode yêu cầu gọi LLM, có thể tốn nhiều thời gian nhất (hàng giây đến hàng chục giây)
- RunCode và ExecuteUnitTests phụ thuộc vào độ phức tạp của code và tests

**Độ phức tạp không gian**:
- error_history lưu trữ tối đa `max_tries` thông báo lỗi
- Mỗi phiên bản của Cφi có thể yêu cầu lưu trữ một lượng lớn mã nguồn
- Kết quả kiểm thử Rφi cũng cần không gian lưu trữ tương ứng với số lượng kiểm thử

**Đánh giá chung**:
- Với các tác vụ phức tạp, chi phí chính là việc gọi LLM (GenerateCode, DebugCode)
- Tham số `max_tries` và `threshold` kiểm soát trực tiếp số lần lặp tối đa và chiến lược thử lại
- Thuật toán có cơ chế tránh lặp vô hạn bằng cách giới hạn số lần thử và phát hiện lỗi lặp lại

## So Sánh Với Các Phương Pháp Khác

### Các Phương Pháp Gỡ Lỗi và Kiểm Thử Truyền Thống

| Phương pháp | Ưu điểm | Nhược điểm |
|-------------|---------|------------|
| **Gỡ lỗi thủ công** | - Kiểm soát cao<br>- Phù hợp với các lỗi phức tạp<br>- Con người có hiểu biết ngữ cảnh | - Tốn thời gian<br>- Phụ thuộc vào kỹ năng<br>- Không thể mở rộng |
| **Kiểm thử đơn vị truyền thống** | - Đảm bảo tính đúng đắn<br>- Tái sử dụng tests<br>- Phát hiện sớm lỗi | - Thời gian viết tests cao<br>- Khó bảo trì<br>- Khó bao phủ mọi trường hợp |
| **Kiểm thử tự động** | - Tiết kiệm thời gian<br>- Thực hiện nhất quán<br>- Có thể lặp lại | - Thiếu sáng tạo<br>- Giới hạn về phạm vi<br>- Khó xử lý trường hợp đặc biệt |
| **Phương pháp LLM cơ bản** | - Tạo code nhanh<br>- Khả năng học từ dữ liệu<br>- Đa dạng giải pháp | - Dễ tạo lỗi<br>- Thiếu cơ chế sửa lỗi<br>- Không đảm bảo chất lượng |
| **Gỡ lỗi và kiểm thử lặp lại (AutoKaggle)** | - Tự động sửa lỗi<br>- Đảm bảo chất lượng với unit tests<br>- Giảm thiểu sự can thiệp của con người<br>- Tạo mới code khi cần | - Phụ thuộc vào chất lượng LLM<br>- Tốn tài nguyên cho nhiều lần gọi LLM<br>- Có thể bị kẹt trong vòng lặp với lỗi phức tạp |

### So Sánh Với Các Framework Khác

**AutoKaggle vs. Các Framework AI Coding Assistant Khác**

| Khía cạnh | AutoKaggle | GitHub Copilot | ML-Copilot | GPT-Engineer |
|-----------|------------|----------------|------------|-------------|
| **Quy trình gỡ lỗi** | Lặp đi lặp lại với kiểm thử đơn vị | Gợi ý sửa lỗi nhưng không lặp lại | Phụ thuộc vào người dùng | Cơ bản, không có kiểm thử |
| **Tự phát hiện lỗi** | Có, thông qua thực thi và kiểm thử | Hạn chế, chủ yếu lỗi cú pháp | Hạn chế | Hạn chế |
| **Kiểm thử tích hợp** | Tích hợp sâu, là một phần của quy trình | Không tích hợp | Tùy chọn | Không tích hợp |
| **Tạo lại code** | Tự động khi phát hiện lỗi lặp lại | Không | Không | Không |
| **Đảm bảo chất lượng** | Cao, qua unit tests và review | Trung bình | Trung bình | Thấp |
| **Phù hợp với các tác vụ khoa học dữ liệu** | Rất cao, được thiết kế riêng | Trung bình | Cao | Thấp-Trung bình |

## Tầm Quan Trọng Của Unit Testing Trong Khoa Học Dữ Liệu

### Lý Do Unit Testing Đặc Biệt Quan Trọng Trong Khoa Học Dữ Liệu

> "In complex and accuracy-demanding tasks like Kaggle data science competitions, merely ensuring that the code runs without errors is not enough. These competitions often involve intricate data processing and sophisticated algorithms, where hidden logical errors can significantly affect the final results. Therefore, it is necessary to design meticulous unit tests that not only verify the correctness of the code but also ensure it meets the expected logical and performance standards." (Section 3.2, Page 5)

Unit testing trong khoa học dữ liệu đóng vai trò đặc biệt quan trọng vì những lý do sau:

1. **Phát Hiện Lỗi Logic Ẩn**:
   - Nhiều lỗi trong khoa học dữ liệu không phải là lỗi cú pháp mà là lỗi logic
   - Ví dụ: Xử lý giá trị thiếu không đúng, phương pháp xử lý ngoại lai không phù hợp, biến đổi đặc trưng không chính xác

2. **Đảm Bảo Tính Nhất Quán Của Quy Trình**:
   - Các giai đoạn trong quy trình khoa học dữ liệu phụ thuộc vào nhau
   - Lỗi ở giai đoạn trước có thể gây ra hậu quả nghiêm trọng ở các giai đoạn sau
   - Ví dụ: "Unnoticed logical defects during the data cleaning phase may lead to poor feature extraction, thereby affecting the model building in subsequent phases." (Section 3.2, Page 5)

3. **Bảo Vệ Khỏi Trường Hợp Đặc Biệt**:
   - Dữ liệu thực tế thường có nhiều trường hợp đặc biệt và ngoại lệ
   - Unit testing giúp xác minh rằng code xử lý được tất cả các trường hợp

4. **Hướng Dẫn Quá Trình Gỡ Lỗi**:
   - Các test thất bại cung cấp thông tin cụ thể về những gì cần sửa
   - Giúp LLM tập trung vào các vấn đề cụ thể thay vì phải phân tích toàn bộ code

### Thiết Kế Unit Tests Trong Khoa Học Dữ Liệu

Theo bài báo, unit tests cho mỗi giai đoạn phải được thiết kế cẩn thận để bao quát nhiều kịch bản, bao gồm cả các trường hợp biên và điểm lỗi tiềm tàng. Cụ thể:

1. **Tests Cho Giai Đoạn Làm Sạch Dữ Liệu**:
   - Kiểm tra xử lý giá trị thiếu
   - Kiểm tra xử lý ngoại lai
   - Kiểm tra chuyển đổi kiểu dữ liệu
   - Kiểm tra xử lý dữ liệu trùng lặp

2. **Tests Cho Giai Đoạn Kỹ Thuật Đặc Trưng**:
   - Kiểm tra mã hóa biến phân loại
   - Kiểm tra tạo đặc trưng mới
   - Kiểm tra chọn lọc đặc trưng
   - Kiểm tra chuẩn hóa/tỷ lệ đặc trưng

3. **Tests Cho Giai Đoạn Xây Dựng Mô Hình**:
   - Kiểm tra chia tập huấn luyện/kiểm thử
   - Kiểm tra hình dạng đầu vào/đầu ra mô hình
   - Kiểm tra đánh giá mô hình
   - Kiểm tra định dạng dự đoán cuối cùng

## Kết Luận

Quy trình gỡ lỗi và kiểm thử lặp lại là một thành phần quan trọng trong framework AutoKaggle, giúp đảm bảo code được tạo ra bởi LLM có chất lượng cao và đáng tin cậy. Quy trình này kết hợp ba công cụ chính: thực thi code, gỡ lỗi code và kiểm thử đơn vị để xác minh cả tính đúng đắn về cú pháp lẫn logic của code.

Những điểm chính cần ghi nhớ:

1. **Quy Trình Lặp Đi Lặp Lại**: Quy trình này không chỉ sửa lỗi một lần mà lặp đi lặp lại cho đến khi code đạt yêu cầu hoặc đến giới hạn số lần thử

2. **Cơ Chế Tạo Lại**: Hệ thống có thể quyết định tạo lại code hoàn toàn nếu phát hiện đang gặp lỗi lặp lại nhiều lần

3. **Kiểm Thử Đơn Vị Toàn Diện**: Kiểm thử không chỉ xác minh code chạy được mà còn đảm bảo nó đáp ứng các yêu cầu cụ thể của giai đoạn

4. **Giới Hạn Lặp Lại**: Quy trình có giới hạn số lần thử để tránh lặp vô hạn trong trường hợp không thể giải quyết lỗi

Việc kết hợp các kỹ thuật này cho phép AutoKaggle tạo ra code chất lượng cao, đáp ứng các yêu cầu phức tạp của các cuộc thi khoa học dữ liệu trên Kaggle.